In [2]:
import matplotlib.pyplot as plt
import sys
import os
import numpy as np
%load_ext autoreload
%autoreload 2

In [3]:
root_dir = os.path.abspath('../')
sys.path.append(root_dir)
sys.path.append(os.path.join(root_dir,'../DPMJET-III-gitlab'))
# print root_dir

pythia_dir = os.path.join(
    os.path.expanduser('~'), 'devel', 'apps', 'pythia8240')
sys.path.append(os.path.join(pythia_dir, 'lib'))

from impy.definitions import *
from impy.constants import *
from impy.kinematics import EventKinematics
from impy.common import impy_config, pdata

In [4]:
event_kinematics = EventKinematics(
#     plab = 20 * GeV,
    ecm = 19.6 * GeV,
#     p1pdg=2212,
#     p2pdg=2212
    nuc1_prop=(197,79), 
    nuc2_prop=(197,79)
)

In [5]:
impy_config["user_frame"] = 'center-of-mass'
impy_config['tau_stable'] = 10e-12
impy_config['pi0_stable'] = True

In [ ]:
#define model here
#list of models: 'SIBYLL23C', 'SIBYLL23', 'SIBYLL21', 'DPMJETIII306', 'DPMJETIII171', 
#            'EPOSLHC','PHOJET112','PHOJET171','URQMD34','PYTHIA8','QGSJET01C',
#            'QGSJETII03','QGSJETII04'
m_list = ['SIBYLL23C', 'SIBYLL23', 'SIBYLL21', 'DPMJETIII306', 'DPMJETIII171', 
            'EPOSLHC','PHOJET112','PHOJET171','URQMD34','PYTHIA8','QGSJET01C',
           'QGSJETII03','QGSJETII04']
for m in m_list:
    gen_m = make_generator_instance(interaction_model_by_tag[m])
    gen_m.init_generator(event_kinematics)

In [ ]:
n_bins = 30
bins = np.linspace(-1,1.,n_bins+1)
grid = 0.5*(bins[1:] + bins[:-1])
widths = (bins[1:] - bins[:-1])

mt_bins = np.linspace(0, 1.8, n_bins+1)
mt_grid = 0.5*(mt_bins[1:] + mt_bins[:-1])
widths_mt = (mt_bins[1:] - mt_bins[:-1])

m_negpions = np.zeros(n_bins)
ymin = 0
ymax = 0.2

eta_bins = np.linspace(-7,7,n_bins+1)
eta_grid = 0.5*(eta_bins[1:] + eta_bins[:-1])
widths_eta = (eta_bins[1:] - eta_bins[:-1])

m_charged_eta = np.zeros(n_bins)

n_events = 10000
norm = 1./float(n_events)

for event in gen_m.event_generator(event_kinematics, n_events):
    event.filter_final_state_centrality(event_kinematics, (0, 0.06))
    m_charged_eta += norm/widths_eta*np.histogram(event.eta,bins=eta_bins)[0]

for event in gen_m.event_generator(event_kinematics, n_events):
    event.filter_final_state_charged()
    m_negpions += norm/widths*np.histogram(np.sqrt(pdata.mass(-211)**2 + event.pt2[(event.p_ids == -211)&(event.y < ymax)&(event.y > ymin)]),bins=bins)[0]

In [ ]:
l = plt.plot(mt_grid, m_negpions, label=m)
plt.xlim(xmin=0)
plt.semilogy()
plt.legend()

In [ ]:
l = plt.plot(eta_grid, m_charged_eta, label=m)
plt.legend()

In [ ]:
for event in gen_m.event_generator(event_kinematics, 3):
    event.filter_final_state_charged()
    for pid, x, y, z, e in zip(event.p_ids, event.px, event.py, event.pz, event.en):
        print '{0:8}: {1:10.2e} {2:10.2e} {3:10.2e} {4:10.2e}'.format(pid, x, y, z, e) 

In [ ]:
tup = (0,1)
print('{0} + {1} = 1'.format(*tup))